In [1]:
import os
from dotenv import load_dotenv

dotenv_path = '.env'
load_dotenv(dotenv_path)

# Print out the environment variables loaded
for key, value in os.environ.items():
    if 'OPENAI' in key:
        print("OpenAI Key founded")
        #print(f"{key}: {value}")

openai_api_key = os.getenv('OPENAI_API_KEY')

#print(f"OpenAI API Key: {openai_api_key}")

OpenAI Key founded


In [6]:
from langchain_openai import OpenAI
from langchain.prompts.prompt import PromptTemplate

In [6]:
my_template = """
Clasifica el sentimiento del siguiente review.
Sentimiento: Es el review positivo, neutro o negativo?

Review: {review}
"""

In [10]:
llm = OpenAI()

In [10]:
my_prompt = PromptTemplate(
    template=my_template,
    input_variables=["review"]
)

In [12]:
preguntaReview1 = my_prompt.format(review="No me gusto el servicio")
print(preguntaReview1)


Clasifica el sentimiento del siguiente review.
Sentimiento: Es el review positivo, neutro o negativo?

Review: No me gusto el servicio



In [14]:
respuestaReview1 = llm.invoke(preguntaReview1)
print(respuestaReview1)

Sentimiento: Negativo


In [15]:
preguntaReview2 = my_prompt.format(review="La verdad que esto fue un buen servicio")
respuestaReview2 = llm.invoke(preguntaReview2)
print(respuestaReview2)


Positivo


### Few Shot Prompting

In [2]:
from langchain.prompts.few_shot import FewShotPromptTemplate

In [18]:
#my_few_shot_template_verbose = """
#Clasifica el sentimiento del siguiente review.
#Sentimiento: Es el review positivo, neutro o negativo?

#Review: {review}

#Ejemplos:
#review: "Eso esta mortal!"
#sentimiento: positivo

#review: "Me estas haciendo perder el tiempo"
#sentimiento: negativo

#review: "Bueno, es mejor que nada"
#sentimiento: neutro
#"""

In [3]:
my_examples = [
    {
        "review": "Eso esta mortal!",
        "response": "sentimiento: positivo" 
    },
    {
        "review": "Me estas haciendo perder el tiempo",
        "response": "sentimiento: negativo" 
    },
    {
        "review": "Bueno, es mejor que nada",
        "response": "sentimiento: neutro" 
    },
]

In [4]:
my_few_shot_template_ok = """
Clasifica el sentimiento del siguiente review.
Sentimiento: Es el review positivo, neutro o negativo?

Review: {review}
{response}
"""

In [7]:
example_prompt = PromptTemplate(
     input_variables=["review", "response"],
     template=my_few_shot_template_ok
 )

In [8]:
my_few_shot_prompt = FewShotPromptTemplate(
    examples=my_examples,
    example_prompt=example_prompt,
    suffix="Review: {review}",
    input_variables=["review"]
)

In [13]:
preguntaReviewFS1 = my_few_shot_prompt.format(review="Ustedes me tienen cansado")
respuestaReviewFS1 = llm.invoke(preguntaReviewFS1)  # Use invoke instead of __call__
print(respuestaReviewFS1)


sentimiento: negativo



In [14]:
preguntaReviewFS2 = my_few_shot_prompt.format(review="Eso esta chulo")
respuestaReviewFS2 = llm.invoke(preguntaReviewFS2)  # Use invoke instead of __call__
print(respuestaReviewFS2)

!
sentimiento: positivo


### Output Parse

In [20]:
from langchain.output_parsers import PydanticOutputParser
from langchain_core.pydantic_v1 import BaseModel, Field, validator
from typing import List

In [21]:
class Reviews_Desired_Output_Structure(BaseModel):
    sentiment: List[str] = Field(
        description="Es positivo, negativo o neutro?"
    )
    positive: List[str] = Field(
        description="Identifica y resume lo positivo descrito en el review?"
    )
    negative: List[str] = Field(
        description="Identifica y resume lo negativo descrito en el review?"
    )

In [22]:
output_parser = PydanticOutputParser(
    pydantic_object=Reviews_Desired_Output_Structure
)

In [23]:
template_for_desired_output_with_parser = """
Este es el review: {big_review}.

Yo quiero como output 3 cosas sobre el review en formato JSON dictionary:

{format_instructions}

"sentiment": Es positivo, negativo o neutro?
"positive": Identifica y resume lo positivo descrito en el review?
"negative": Identifica y resume lo negativo descrito en el review?

"""

In [24]:
prompt_template_with_parser = PromptTemplate(
    template=template_for_desired_output_with_parser,
    input_variables=["big_review"],
    partial_variables={
        "format_instructions": output_parser.get_format_instructions()
    }
)

In [25]:
user_input = """
La verdad que visité la Plaza de la Cultura,.. y en este momento solo esta funcionando el Museo de Historia y Geografia , lo cual me pareció muy bien remodelado.
El Teatro Nacional cuando hay obras de teatro lo encuentro bien, solo que deberian poner mas las reglas de que las personas vayan mejor vestidas.
El Museo de Arte Moderno.. lo estan remodelando.. ojalá terminen pronto y que la espera en la remodelacion valga la pena.. le hacia falta al lugar.
Lo que no me gustó fué que desbarataron unos murales un artista Dominicano y eso no pudo pasar.
"""

In [26]:
question = prompt_template_with_parser.format(big_review=user_input)

In [27]:
response = llm(question)

In [28]:
formatted_output = output_parser.parse(response)

In [29]:
print(type(formatted_output))

<class '__main__.Reviews_Desired_Output_Structure'>


In [30]:
import json
json_output = formatted_output.json()
python_dict = json.loads(json_output)
print(python_dict)

{'sentiment': ['neutral'], 'positive': ['La Plaza de la Cultura está bien remodelada.'], 'negative': ['La desaparición de los murales del artista dominicano es algo negativo.']}


In [31]:
print(type(python_dict))

<class 'dict'>
